In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch
from constants import regions

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#Import sklearn modules
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [ ]:
### Define the functions that I will use (paste them from the another notebook

## Comment out the bottom two lines
sys.argv[1] = 1
sys.argv[2] = 2

In [7]:
# d_type_num defines the data type using an integer 0,1,2,3, or 4 
d_type_num = int(sys.argv[1])
data_type = ['Wake', 'N1', 'N2', 'N3','REM'][d_type_num]

expt_num = int(sys.argv[2])

In [ ]:
t1 = time.time()

reduced_slide_list = [ (0.5, 0.5), (1, 0.5), (2, 0.5), (3, 0.5), (4, 0.5), (6, 0.5), (8, 0.5), (10, 1), (12, 1), (14, 1)]
super_reduced_slide_list = [ (0.5, 0.5), (1, 0.5), (2, 0.5), (4, 1), (6, 1), (8, 1), (10,1), (14, 1) ] 



#2. Load the data based on data type ___________________________________________________________
df = pd.read_hdf(os.path.join(os.path.abspath('..'),'Multiple_PSDs_Per_Participant/' + data_type + 'processed_multiple_psds_per_person.h5') , key = 'df')

y = df['y'].copy()
groups = df['groups'].copy()

df = df.drop(columns = ['groups', 'y'])

regional_dfs = {}

for region in regions:
    
    single_region_df = df[[col for col in df.columns if col.endswith('_' + region)]]
    frequency_vals  = np.arange(0.5,40.125, 0.125)
    single_region_df.columns = frequency_vals
    
    folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'
    # paths = joblib.load(folder + data_type + '_paths.pkl')
    
    # groups = pd.Series(paths['s_p_id'])
    # s_class_list = pd.Series(paths['s_class_list'])
    # y = s_class_list.map({'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3})
    
    #3 Generate the frequency band search space ____________________________________________________
    width_slide_list = [(0.5,0.5)] 
    for window_width in range(1, 15 ):
        width_slide_list.append((window_width, 0.5) )
    
    #4 Calculate all of the features ________________________________________________________________
    feature_dfs = []
    
    for window_width_hz, window_slide_hz in super_reduced_slide_list:
        description = 'window width : ' + str(window_width_hz) + ' .  window stride : ' + str(window_slide_hz)
        
        #1.Set Window width in Hz and Window Slide in Hz
        # window_width_hz = 3
        window_len = (window_width_hz/0.125)+1
        assert window_len%1 == 0
        window_len = int(window_len)
        
        # window_slide_hz = 0.5 
        window_slide_len = window_slide_hz/0.125 
        assert window_slide_len%1 == 0
        window_slide_len = int(window_slide_len)
        
        
        #2.Calculate band values and store in dataframe calculated_df
        
        # Start of the window is the middle_freq value minus window_width_hz/2 
        # End of the window is the middle freq value plus window_width_hz/2
        window_len
        
        middle_freq = []
        final_cols = []
        
        i = 0
        while i < len(frequency_vals) - window_len:
            middle_freq.append( frequency_vals[i:i+window_len].mean() )
            band_vals = single_region_df.iloc[:,i:i+window_len].mean(axis = 1) #For every row calculate the mean for the appropriate elements
            
            final_cols.append(band_vals)
            
            # scaled_psd.append( psd_values[i:i+factor].mean() )
            i+= window_slide_len
        
        calculated_df = pd.DataFrame(final_cols).T
        calculated_df.columns = middle_freq
        
        calculated_df.columns = [ str(col) + '_width_' + str(window_width_hz) for col in calculated_df.columns]
        
        
        #3.Calculate Information Gain Based on Features
        #Change to binary HC vs PD / PD+RBD ---> Generate expt specific X,y,groups 
        X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(calculated_df,y,groups,expt_num)
    
        feature_dfs.append(X_expt)
            
    #5 Concatenate all feature_dfs into a single dataframe_________________________________________________________________
    X_expt_concatenated = pd.concat(feature_dfs, axis=1)

    #6 Calculate Frequency Ratio Columns
    calculated_cols = []
    col_names = []
    
    X_expt_ratios = X_expt_concatenated.copy() 
    
    for col_1 in X_expt_concatenated.columns:
        for col_2 in X_expt_concatenated.columns:
            if col_1 == col_2:
                pass
            else:
                new_col = col_1 + '/' + col_2  
                col_names.append(new_col)
                calculated_cols.append( X_expt_concatenated[col_1]/X_expt_concatenated[col_2] )
    
    df_ratios = pd.concat(calculated_cols, axis = 1)
    df_ratios.columns = col_names
    df_ratios_and_freq = pd.concat([df_ratios, X_expt_concatenated], axis = 1)
    regional_dfs[region] = df_ratios_and_freq.copy()
    print(region + 'done!')
    t2 = time.time()
    time_dif = t2 - t1
    print(time_dif)

#### Experiments loop

In [9]:
groups = groups_expt.copy()
X = X_expt.copy()
y = y_expt.copy()

In [ ]:
for region in regions:
    X_expt = regional_dfs[region]
    X = X_expt.copy()
    groups = groups_expt.copy()
    y = y_expt.copy()


    num = 1
    gkf = GroupKFold(n_splits = 5) 
    pred_arrays = []
    test_arrays = []
    
    for train_index, test_index  in gkf.split(X, y, groups = groups.astype(int)*num):
                        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
                        #Fit model 
                        DT1, cols_used_in_DT1, DT2, cols_used_in_DT2 = fit_model(X_train, y_train , thresh = 0.9)
                        
                        #Make model predictions
                        final_preds_array = model_predict(DT1, cols_used_in_DT1, DT2,cols_used_in_DT2, X_test, thresh = 0.9)
    
                        #Append pred arrays and test arrays to list 
                        pred_arrays.append(final_preds_array)
                        test_arrays.append(y_test)
                        print('done another juan ...!')
    
    
    final_preds = np.concatenate(pred_arrays)
    final_test = np.concatenate(test_arrays)
    
    correct_ratio , percent_preds_made = model_acc(final_preds, final_test)

In [10]:
np.random.rand(10)

array([0.14371265, 0.99809292, 0.89597699, 0.76866771, 0.74530679,
       0.29568417, 0.42567583, 0.01884471, 0.83698891, 0.85239958])

In [11]:
results_dict = {}
for region in regions:
    results_dict[region] = {}
    results_dict[region]['preds'] = np.random.rand(10) 
    results_dict[region]['test'] = np.random.rand(10)

In [13]:
joblib.dump(results_dict, 'test_results_dictionary.pkl')

['test_results_dictionary.pkl']

In [14]:
joblib.load('test_results_dictionary.pkl')

{'Prefrontal': {'preds': array([0.6787853 , 0.60212086, 0.17663649, 0.4661643 , 0.77255717,
         0.67274208, 0.29417726, 0.47232595, 0.39705857, 0.54070729]),
  'test': array([0.47039858, 0.49903648, 0.2306049 , 0.32057108, 0.90686427,
         0.95776997, 0.0043754 , 0.51401969, 0.14357803, 0.04094011])},
 'Frontal': {'preds': array([0.65539544, 0.39043158, 0.11763664, 0.60416844, 0.48133576,
         0.28055749, 0.75546095, 0.65184154, 0.44551991, 0.22187223]),
  'test': array([0.15108675, 0.44458813, 0.65045314, 0.73266002, 0.43949984,
         0.04896986, 0.372002  , 0.14719808, 0.78645783, 0.79068293])},
 'Left Frontal': {'preds': array([0.75469605, 0.9706576 , 0.45984878, 0.94604433, 0.52647082,
         0.4919449 , 0.34779677, 0.76345201, 0.47446961, 0.9745487 ]),
  'test': array([0.47297537, 0.53963175, 0.40072497, 0.08923634, 0.66909952,
         0.24984612, 0.6114832 , 0.65461352, 0.93380268, 0.26543658])},
 'Right Frontal': {'preds': array([0.11940888, 0.42133589, 0.6039